You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [1]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [2]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [3]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [4]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [5]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [6]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [7]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [8]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [9]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [10]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan on Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of latest trends and key players in the AI sector

Key Points:
1. Latest Trends in AI:
- Advancements in machine learning algorithms
- Increased adoption of AI in healthcare, finance, and retail sectors
- Emergence of autonomous vehicles and smart assistants

2. Key Players in the AI Industry:
- Google, Microsoft, and Amazon leading in AI research and developm

I now can give a great answer

Final Answer:
# The Rise of Artificial Intelligence: Trends, Players, and Ethical Concerns

## Introduction

Artificial Intelligence (AI) has become a driving force in various industries, revolutionizing the way businesses operate and transforming the way we live. From advancements in machine learning algorithms to the emergence of autonomous vehicles and smart assistants, AI continues to shape the future of technology. Leading players in the AI industry, such as Google, Microsoft, and Amazon, are at the forefront of research and development, while startups like OpenAI and UiPath are making significant contributions to innovation. Collaboration between academia and industry further propels AI advancements, creating a dynamic landscape of possibilities.

## Latest Trends in AI

In recent years, AI has witnessed significant advancements, particularly in machine learning algorithms. These algorithms are becoming more sophisticated, enabling AI systems to lea

- Display the results of your execution as markdown in the notebook.

In [11]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: Trends, Players, and Ethical Concerns

## Introduction

Artificial Intelligence (AI) has become a driving force in various industries, revolutionizing the way businesses operate and transforming the way we live. From advancements in machine learning algorithms to the emergence of autonomous vehicles and smart assistants, AI continues to shape the future of technology. Leading players in the AI industry, such as Google, Microsoft, and Amazon, are at the forefront of research and development, while startups like OpenAI and UiPath are making significant contributions to innovation. Collaboration between academia and industry further propels AI advancements, creating a dynamic landscape of possibilities.

## Latest Trends in AI

In recent years, AI has witnessed significant advancements, particularly in machine learning algorithms. These algorithms are becoming more sophisticated, enabling AI systems to learn and adapt in real-time. Moreover, the adoption of AI in sectors like healthcare, finance, and retail is on the rise, with organizations leveraging AI to enhance decision-making processes and improve customer experiences. The development of autonomous vehicles and smart assistants also showcases the growing capabilities of AI in transforming industries and everyday life.

## Key Players in the AI Industry

The AI industry is dominated by tech giants like Google, Microsoft, and Amazon, who are leading the way in AI research and development. These companies are investing heavily in AI technologies, driving innovation and shaping the future of AI. Additionally, startups such as OpenAI and UiPath are making significant contributions to the field, with disruptive technologies that are redefining industry standards. The collaboration between academia and industry further accelerates AI innovation, fostering a culture of creativity and exploration.

## Noteworthy News on AI

While AI presents numerous opportunities for growth and innovation, it also raises ethical concerns surrounding privacy, bias, and accountability. Regulatory developments in AI governance aim to address these concerns, ensuring that AI technologies are developed and deployed responsibly. Moreover, the impact of AI on job displacement and workforce transformation has sparked discussions about the future of work in an AI-driven world. As AI continues to evolve, it is essential to consider the societal implications and ethical considerations associated with AI technologies.

In conclusion, Artificial Intelligence is a transformative force that is reshaping industries, driving innovation, and raising important ethical considerations. As AI continues to advance, it is crucial for businesses, researchers, and policymakers to stay informed and engaged with the latest trends and developments in the field. By exploring AI courses and certifications, individuals can prepare for the future of work in an AI-powered world, contributing to the growth and evolution of AI technology. Stay updated on AI news and developments, and join the conversation on the impact of AI on society and business.

# Call to Action

Stay updated on the latest AI news and developments to stay ahead of the curve in the ever-evolving field of Artificial Intelligence. Explore AI courses and certifications to enhance your skills and prepare for the future of work in an AI-driven world. Join the conversation on ethical AI concerns, regulatory developments, and the impact of AI on jobs, and be a part of the transformative journey of Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [12]:
topic = "Agentic AI for AEC Industry"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Agentic AI for AEC Industry.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document:

Title: Harnessing the Power of Agentic AI in the AEC Industry

Introduction:
- Brief overview of the increasing importance of Agentic AI in the AEC industry
- Introduction of key trends, key players, and recent news in the field

Key Points:
1. Latest Trends in Agentic AI for AEC Industry:
- Integration of AI-powered tools for improved project management and efficiency
- Utilization of AI in design optimization and predictive analytics
- Implementation of AI-driven solutions for ris

I now can give a great answer

Final Answer:
# **Harnessing the Power of Agentic AI in the AEC Industry**

## Introduction
The Architecture, Engineering, and Construction (AEC) industry is currently undergoing a significant transformation due to the widespread adoption of Agentic AI. This groundbreaking technology is revolutionizing aspects of project management, design optimization, and risk management within the field. With the continuous evolution of AI, it is imperative for industry professionals to remain well-informed about the latest developments, key players, and recent advancements in Agentic AI for AEC.

## Latest Trends in Agentic AI for AEC Industry
A prominent trend in the AEC sector is the integration of AI-powered tools to enhance project management processes and overall efficiency. Through the utilization of AI algorithms, companies can streamline workflows, automate repetitive tasks, and enhance decision-making capabilities. Furthermore, AI is being leveraged for desig

In [13]:
Markdown(result)

# **Harnessing the Power of Agentic AI in the AEC Industry**

## Introduction
The Architecture, Engineering, and Construction (AEC) industry is currently undergoing a significant transformation due to the widespread adoption of Agentic AI. This groundbreaking technology is revolutionizing aspects of project management, design optimization, and risk management within the field. With the continuous evolution of AI, it is imperative for industry professionals to remain well-informed about the latest developments, key players, and recent advancements in Agentic AI for AEC.

## Latest Trends in Agentic AI for AEC Industry
A prominent trend in the AEC sector is the integration of AI-powered tools to enhance project management processes and overall efficiency. Through the utilization of AI algorithms, companies can streamline workflows, automate repetitive tasks, and enhance decision-making capabilities. Furthermore, AI is being leveraged for design optimization and predictive analytics, empowering architects and engineers to develop innovative solutions and anticipate project outcomes with increased precision. Additionally, AI-driven solutions are being implemented for risk management and safety improvements, leading to a reduction in potential hazards and an enhancement of project quality.

## Key Players in the Agentic AI for AEC Industry
Notable industry leaders like Autodesk, Bentley Systems, and Trimble are spearheading the development of AI solutions tailored specifically for the AEC industry. These established companies are harnessing AI technologies to create cutting-edge tools for project visualization, simulation, and analysis. Moreover, emerging startups focusing on AI applications for architecture, engineering, and construction are challenging conventional practices and fostering innovation in the field. Collaborative efforts between technology giants and AEC firms are paving the way for the progression of AI technology within the industry, resulting in more efficient project delivery and heightened client satisfaction.

## Noteworthy News in Agentic AI for AEC Industry
Recent case studies have highlighted the successful integration of AI in various AEC projects, showcasing the tangible benefits of Agentic AI in real-world scenarios. Regulatory updates are also playing a crucial role in shaping the adoption of AI within the construction sector, with the establishment of guidelines and standards to ensure the ethical and safe utilization of AI technologies. Future projections indicate a substantial growth and influence of Agentic AI in the AEC industry, with AI set to become an essential tool for fostering innovation and efficiency in project execution.

In conclusion, the incorporation of Agentic AI in the AEC industry is transforming traditional practices and reshaping the future of construction. By harnessing the capabilities of AI technologies, industry professionals can optimize project workflows, mitigate risks, and deliver high-quality projects with enhanced efficiency. As the AEC sector continues to embrace AI solutions, it is vital for professionals to stay abreast of the evolving landscape of Agentic AI for sustainable growth and success.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

In [14]:
topic = "Project Management As A Service : tailored for the AEC industry personas"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Project Management As A Service : tailored for the AEC industry personas.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Future of Project Management As A Service in the AEC Industry: Trends, Players, and Opportunities

Introduction:
- Brief overview of Project Management As A Service (PMaaS) in the Architecture, Engineering, and Construction (AEC) industry
- Importance of tailored project management solutions for AEC personas
- Purpose of the article to explore the latest trends, key players, and news in PMaaS for the AEC industry

Key Points:
1. Late

I now can give a great answer

Final Answer: 

# The Future of Project Management As A Service in the AEC Industry: Trends, Players, and Opportunities

## Introduction

Project Management As A Service (PMaaS) is a crucial component of the Architecture, Engineering, and Construction (AEC) industry, providing tailored project management solutions for various stakeholders. With the AEC industry undergoing digital transformation, the demand for efficient project management tools and services has never been higher. This article delves into the latest trends, key players, and opportunities in PMaaS for the AEC industry, offering insights into the advancements shaping the future of project management in this sector.

## Latest Trends in PMaaS for AEC

A prominent trend in PMaaS for the AEC industry is the widespread adoption of cloud-based project management platforms. These platforms offer flexibility, scalability, and real-time access to project data, facilitating seamless collaboration amo

In [15]:
Markdown(result)

# The Future of Project Management As A Service in the AEC Industry: Trends, Players, and Opportunities

## Introduction

Project Management As A Service (PMaaS) is a crucial component of the Architecture, Engineering, and Construction (AEC) industry, providing tailored project management solutions for various stakeholders. With the AEC industry undergoing digital transformation, the demand for efficient project management tools and services has never been higher. This article delves into the latest trends, key players, and opportunities in PMaaS for the AEC industry, offering insights into the advancements shaping the future of project management in this sector.

## Latest Trends in PMaaS for AEC

A prominent trend in PMaaS for the AEC industry is the widespread adoption of cloud-based project management platforms. These platforms offer flexibility, scalability, and real-time access to project data, facilitating seamless collaboration among project participants. Moreover, the integration of Building Information Modeling (BIM) technology with project management tools has gained traction, enabling more effective project planning, design, and construction processes. Another noteworthy trend is the focus on real-time collaboration and communication among project stakeholders, enhancing decision-making and issue resolution throughout the project lifecycle.

## Key Players in PMaaS for AEC

Numerous leading project management service providers cater to the AEC industry, delivering customized solutions to meet the unique requirements of architects, engineers, and construction experts. These providers often showcase case studies and success stories of organizations that have successfully utilized PMaaS to streamline their project workflows and boost efficiency. By spotlighting these key players, AEC firms can make informed decisions when selecting project management services that align with their project needs and business goals.

## Noteworthy News in PMaaS for AEC

Recent advancements and innovations in project management tools tailored for AEC projects continue to shape the industry landscape. These developments aim to address the distinct challenges and demands of AEC projects, including managing complex projects with multiple stakeholders and ensuring compliance with industry regulations and standards. Industry regulations and standards significantly influence PMaaS implementation, prompting organizations to adopt solutions that comply with legal requirements and best practices in project management.

By staying abreast of the latest trends, key players, and news in PMaaS for the AEC industry, stakeholders can gain valuable insights and opportunities for professional advancement. Through effective project management strategies and the integration of innovative project management tools and services, AEC firms can streamline their project processes, enhance collaboration among project participants, and achieve greater success in their projects.

In [16]:
topic = "AI-infused PMaaS for AEC industry ecosystem"
result = crew.kickoff(inputs={"topic": topic})



 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI-infused PMaaS for AEC industry ecosystem.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: Unleashing the Power of AI-infused PMaaS in the AEC Industry Ecosystem

Introduction:
- Brief overview of AI-infused Project Management as a Service (PMaaS) and its impact on the Architecture, Engineering, and Construction (AEC) industry.
- Introduction to the latest trends, key players, and noteworthy news in AI-infused PMaaS for the AEC industry ecosystem.

Key Points:
1. Latest Trends in AI-infused PMaaS for AEC:
- Integration of AI technologies like machi

I now can give a great answer
Final Answer: 

# Unleashing the Power of AI-infused PMaaS in the AEC Industry Ecosystem

## Introduction

In today's rapidly evolving Architecture, Engineering, and Construction (AEC) industry, the integration of Artificial Intelligence (AI) into Project Management as a Service (PMaaS) is revolutionizing the way projects are planned, executed, and monitored. AI-powered tools such as machine learning and predictive analytics are enabling project managers to make data-driven decisions, automate scheduling processes, optimize resource allocation, and mitigate risks effectively. This blog post delves into the latest trends, key players, and noteworthy news in AI-infused PMaaS for the AEC industry ecosystem, offering valuable insights for AEC professionals, project managers, and industry stakeholders.

## Latest Trends in AI-infused PMaaS for AEC

The AEC industry is witnessing a significant shift towards the adoption of AI technologies in project management p

In [17]:
Markdown(result)


# Unleashing the Power of AI-infused PMaaS in the AEC Industry Ecosystem

## Introduction

In today's rapidly evolving Architecture, Engineering, and Construction (AEC) industry, the integration of Artificial Intelligence (AI) into Project Management as a Service (PMaaS) is revolutionizing the way projects are planned, executed, and monitored. AI-powered tools such as machine learning and predictive analytics are enabling project managers to make data-driven decisions, automate scheduling processes, optimize resource allocation, and mitigate risks effectively. This blog post delves into the latest trends, key players, and noteworthy news in AI-infused PMaaS for the AEC industry ecosystem, offering valuable insights for AEC professionals, project managers, and industry stakeholders.

## Latest Trends in AI-infused PMaaS for AEC

The AEC industry is witnessing a significant shift towards the adoption of AI technologies in project management processes. AI-powered tools are revolutionizing project planning by providing real-time data analysis and decision-making capabilities. Machine learning algorithms are being utilized for automated scheduling, resource allocation, and risk management, leading to improved project efficiency and outcomes. The integration of AI in project management is not just a trend but a necessity for AEC companies looking to stay competitive in a rapidly changing landscape.

## Key Players in AI-infused PMaaS for AEC

Leading companies in the AEC sector are investing in AI-powered project management solutions to streamline their operations and enhance project delivery. Case studies showcasing successful implementation of AI-infused PMaaS by industry giants highlight the tangible benefits of leveraging AI technologies in project management. Collaborations between tech firms, AEC companies, and research institutions are driving innovation in AI-infused project management, paving the way for future advancements in the industry.

## Noteworthy News in AI-infused PMaaS for AEC

Recent developments in AI technologies are reshaping the future of project management in the AEC industry. Success stories of AEC firms utilizing AI-infused PMaaS to optimize project workflows and achieve better project outcomes are inspiring industry professionals to explore the potential of AI in their own projects. Industry events, conferences, and webinars focusing on AI applications in project management are providing valuable insights and networking opportunities for AEC professionals looking to stay ahead of the curve.

In conclusion, AI-infused PMaaS is a game-changer for the AEC industry, offering unprecedented opportunities for innovation, efficiency, and cost savings. By embracing AI technologies in project management, AEC professionals can unlock new possibilities for project success and drive sustainable growth in the industry.

**Call to Action:** Explore AI-infused PMaaS solutions for your project management needs, share your experiences and insights on AI adoption in the AEC industry, and access relevant resources and tools for further learning and implementation.

By following this comprehensive content plan, the blog article on AI-infused PMaaS for the AEC industry ecosystem aims to provide valuable insights, actionable information, and engaging content for our target audience.